In [2]:
# install necessary libraries
import os, random, time
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [3]:
class CFG:
    seed = 42
    n_folds = 5
    img_size = 224
    batch_size = 16
    epochs = 12
    model_name = 'tf_efficientnet_b4_ns'
    lr = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-5
    train_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train'
    test_dir  = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test'
    train_csv = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv'
    test_csv  = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv'
    out_dir   = './'
    device    = 'cuda' if torch.cuda.is_available() else 'cpu'
    amp       = True

In [8]:
# seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if CFG.device=='cuda':
        torch.cuda.manual_seed_all(seed)

set_seed(CFG.seed)
print("Device:", CFG.device)

Device: cuda


In [9]:
# data augmentation
train_aug = A.Compose([
    A.RandomResizedCrop(size=(CFG.img_size, CFG.img_size), scale=(0.75, 1.0), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2, p=0.5),
    A.Affine(translate_percent=0.0625, scale=(0.9, 1.1), rotate=15, p=0.5),
    A.CLAHE(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

valid_aug = A.Compose([
    A.Resize(CFG.img_size, CFG.img_size),
    A.Normalize(),
    ToTensorV2()
])

In [15]:
# Input data structure
class SoilDataset(Dataset):
    def __init__(self, df, img_dir, transforms=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms
        self.is_test = is_test

        # Build stem -> filename map once
        self.stem2file = {}
        for fname in os.listdir(self.img_dir):
            stem, ext = os.path.splitext(fname)
            self.stem2file[stem] = fname

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        raw_id = str(row["image_id"]).strip()

        # Strip any extension from the CSV image_id
        img_stem = os.path.splitext(raw_id)[0]

        # Lookup the real filename
        fname = self.stem2file.get(img_stem)
        if fname is None:
            raise FileNotFoundError(f"No file found for image_id={raw_id} (stem={img_stem}) in {self.img_dir}")

        img_path = os.path.join(self.img_dir, fname)
        img = np.array(Image.open(img_path).convert("RGB"))
        if self.transforms:
            img = self.transforms(image=img)["image"]

        if self.is_test:
            return img, raw_id

        label = torch.tensor(row["label"], dtype=torch.float32)
        return img, label

In [17]:
# training loop
def train_one_epoch(model, loader, optimizer, criterion, scaler, epoch):
    model.train()
    losses = []
    pbar = tqdm(loader, desc=f"Epoch {epoch} [Train]", leave=False)
    for imgs, labels in pbar:
        imgs, labels = imgs.to(CFG.device), labels.to(CFG.device).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=CFG.amp):
            logits = model(imgs)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        losses.append(loss.item())
        pbar.set_postfix(loss=f"{np.mean(losses):.4f}", lr=f"{optimizer.param_groups[0]['lr']:.1e}")

    return np.mean(losses)


# validation loop
@torch.no_grad()
def validate(model, loader, criterion, epoch):
    model.eval()
    losses, preds, targs = [], [], []
    pbar = tqdm(loader, desc=f"Epoch {epoch} [Valid]", leave=False)
    for imgs, labels in pbar:
        imgs, labels = imgs.to(CFG.device), labels.to(CFG.device).unsqueeze(1)
        logits = model(imgs)
        loss = criterion(logits, labels)

        losses.append(loss.item())
        probs = torch.sigmoid(logits).cpu().numpy().flatten()
        preds += probs.tolist()
        targs += labels.cpu().numpy().flatten().tolist()

        f1 = f1_score(targs, np.array(preds) > 0.5)
        pbar.set_postfix(val_loss=f"{np.mean(losses):.4f}", val_f1=f"{f1:.4f}")

    return np.mean(losses), f1

In [20]:
# k-fold cross validation implementation
df = pd.read_csv(CFG.train_csv)
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
test_df = pd.read_csv(CFG.test_csv)

# accumulate test predictions for fold-averaging
test_preds = np.zeros(len(test_df))

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df.label)):
    print(f"\n>>> Fold {fold+1}/{CFG.n_folds}")
    train_df = df.loc[train_idx]
    val_df   = df.loc[val_idx]

    train_loader = DataLoader(SoilDataset(train_df, CFG.train_dir, train_aug),
                              batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(SoilDataset(val_df,   CFG.train_dir, valid_aug),
                              batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    # Model & optimizer per fold
    model = timm.create_model(CFG.model_name, pretrained=True, num_classes=1)
    model.to(CFG.device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    total_steps = CFG.epochs * len(train_loader)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=CFG.lr, total_steps=total_steps,
        pct_start=0.1, anneal_strategy='cos', final_div_factor=CFG.lr/CFG.min_lr
    )
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.amp)

    best_f1 = 0.0
    best_path = os.path.join(CFG.out_dir, f"best_fold{fold}.pth")

    for epoch in range(1, CFG.epochs+1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler, epoch)
        val_loss, val_f1 = validate(model, val_loader, criterion, epoch)
        print(f"Fold {fold} Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_f1={val_f1:.4f}")
        best_f1 = val_f1
        torch.save(model.state_dict(), best_path)
        scheduler.step()

    # Load best and predict test
    model.load_state_dict(torch.load(best_path))
    test_loader = DataLoader(SoilDataset(test_df, CFG.test_dir, valid_aug, is_test=True),
                             batch_size=CFG.batch_size, shuffle=False, num_workers=4)
    preds_fold = []
    for imgs, ids in test_loader:
        imgs = imgs.to(CFG.device)
        with torch.no_grad():
            logits = model(imgs)
            preds_fold += list(torch.sigmoid(logits).cpu().numpy().flatten())
    test_preds += np.array(preds_fold) / CFG.n_folds


>>> Fold 1/5


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_35/1919859290.py:29: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=CFG.amp)


Epoch 1 [Train]:   0%|          | 0/62 [00:00<?, ?it/s]

/tmp/ipykernel_35/2567921498.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=CFG.amp):


Epoch 1 [Valid]:   0%|          | 0/16 [00:00<?, ?it/s]

...

Fold 4 Epoch 11: train_loss=0.0035, val_loss=0.0061, val_f1=1.0000


Epoch 12 [Train]:   0%|          | 0/62 [00:00<?, ?it/s]

/tmp/ipykernel_35/2567921498.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=CFG.amp):


Epoch 12 [Valid]:   0%|          | 0/16 [00:00<?, ?it/s]

Fold 4 Epoch 12: train_loss=0.0032, val_loss=0.0048, val_f1=1.0000
